<a href="https://colab.research.google.com/github/Flychuban/Comment-Toxicity-App/blob/main/Comment_toxicity_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow tensorflow-gpu pandas matplotlib sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Comment_toxicity_folder/comment_toxicity_train.csv')

In [ ]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
from tensorflow.keras.layers import TextVectorization

In [4]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [5]:
MAX_FEATURES = 200000 #number words in the vocab

In [6]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES, output_sequence_length=1800, output_mode='int')

In [7]:
vectorizer.adapt(X.values)

In [ ]:
vectorizer.get_vocabulary()

In [8]:
vectorized_text = vectorizer(X.values)

In [9]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [11]:
batch_X, batch_y = dataset.as_numpy_iterator().next()

In [12]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [ ]:
train.as_numpy_iterator().next()

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [15]:
from pandas.core.algorithms import mode
model = Sequential()
model.add(Embedding(MAX_FEATURES + 1, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

In [16]:
from keras.engine.training import optimizer
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [ ]:
history = model.fit(train, epochs=5, validation_data=val)

Epoch 1/5
6981/6981 [==============================] - 818s 116ms/step - loss: 0.0633 - val_loss: 0.0501
Epoch 2/5
6981/6981 [==============================] - 658s 94ms/step - loss: 0.0464 - val_loss: 0.0412
Epoch 3/5
6981/6981 [==============================] - 654s 94ms/step - loss: 0.0408 - val_loss: 0.0360
Epoch 4/5
6981/6981 [==============================] - 645s 92ms/step - loss: 0.0367 - val_loss: 0.0325
Epoch 5/5
6981/6981 [==============================] - 649s 93ms/step - loss: 0.0330 - val_loss: 0.0292


In [18]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

In [ ]:
model.save_weights('./checkpoints/my_checkpoint')

In [ ]:
model.save("./model_comment_toxicity.h5")

In [21]:
model.load_weights("/content/drive/MyDrive/Comment_toxicity_folder/model_comment_toxicity.h5")

In [22]:
input_text = vectorizer("You freaking suck!")

In [23]:
batch = test.as_numpy_iterator().next()

In [25]:
res = model.predict(np.expand_dims(input_text, 0))
res

1/1 [==============================] - 0s 53ms/step


array([[0.9994966 , 0.12450563, 0.97822434, 0.00540834, 0.94779485,
        0.05023937]], dtype=float32)

In [26]:
from tensorflow.keras.metrics import Precision, CategoricalAccuracy, Recall

In [27]:
pre = Precision()
acc = CategoricalAccuracy()
re = Recall()

In [ ]:
for batch in test.as_numpy_iterator():
  X_true, y_true = batch
  yhat = model.predict(X_true)

  y_true = y_true.flatten()
  yhat = yhat.flatten()

  pre.update_state(y_true, yhat)
  re.update_state(y_true, yhat)
  acc.update_state(y_true, yhat)

In [31]:
print(f"Precision: {pre.result().numpy()} Recall: {re.result().numpy()} Accuracy: {acc.result().numpy()}")

Precision: 0.8551374077796936 Recall: 0.8194047212600708 Accuracy: 0.5035105347633362


In [ ]:
!pip install gradio jinja2

In [33]:
import gradio as gr

In [36]:
input_str = vectorizer("I freaken hate you! I am coming to hunt you down. I am coming for you!")
res = model.predict(np.expand_dims(input_str, 0))
res

1/1 [==============================] - 0s 56ms/step


array([[0.5763147 , 0.00099139, 0.01655209, 0.01056462, 0.10462938,
        0.02322125]], dtype=float32)

In [41]:
def score_comment(comment):
  vectorized_comment = vectorizer([comment])
  results = model.predict(vectorized_comment)

  text = ''
  for idx, col in enumerate(df.columns[2:]):
    text += f'{col}: {results[0][idx] > 0.5}' + '\n'
  return text

In [ ]:
interface = gr.Interface(fn=score_comment,
                         inputs=gr.inputs.Textbox(lines=2, placeholder="Comment to score"),
                         outputs='text')

In [43]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fbad73ca242863ea6f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
